<a href="https://colab.research.google.com/github/maminseo/minseo/blob/main/Algorithms_5thHW_Class2_2243359_MinseoMa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import defaultdict

class MaxFlowMinCut:
    def __init__(self):
        self.capacity = defaultdict(lambda: defaultdict(int))
        self.graph = defaultdict(list)

    def add_edge(self, u, v, c):
        self.capacity[u][v] += c

        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    def _dfs(self, s, t, parent):
        visited = set([s])
        stack = [s]

        while stack:
            u = stack.pop()
            for v in self.graph[u]:
                if v not in visited and self.capacity[u][v] > 0:
                    visited.add(v)
                    parent[v] = u
                    if v == t:
                        return True
                    stack.append(v)
        return False

    def ford_fulkerson(self, s, t):
        parent = {}
        max_flow = 0

        while True:
            parent = {s: None}
            found = self._dfs(s, t, parent)
            if not found:
                break

            path_flow = float('inf')
            v = t
            while v != s:
                u = parent[v]
                path_flow = min(path_flow, self.capacity[u][v])
                v = u

            v = t
            while v != s:
                u = parent[v]
                self.capacity[u][v] -= path_flow
                self.capacity[v][u] += path_flow
                v = u

            max_flow += path_flow

        visited = set([s])
        stack = [s]
        while stack:
            u = stack.pop()
            for v in self.graph[u]:
                if v not in visited and self.capacity[u][v] > 0:
                    visited.add(v)
                    stack.append(v)

        S = visited

        all_vertices = set(self.graph.keys())
        for u in list(self.graph.keys()):
            all_vertices.update(self.graph[u])
        T = all_vertices - S

        return max_flow, S, T


In [ ]:
from collections import deque, defaultdict

def edmonds_karp(capacity, adj, source, sink):

    flow = defaultdict(lambda: defaultdict(int))
    max_flow = 0

    while True:

        parent = {source: None}
        q = deque([source])

        while q and sink not in parent:
            u = q.popleft()
            for v in adj[u]:
                residual = capacity[u][v] - flow[u][v]
                if residual > 0 and v not in parent:
                    parent[v] = u
                    q.append(v)

        if sink not in parent:
            break

        bottleneck = float('inf')
        v = sink
        while parent[v] is not None:
            u = parent[v]
            bottleneck = min(bottleneck, capacity[u][v] - flow[u][v])
            v = u

        v = sink
        while parent[v] is not None:
            u = parent[v]
            flow[u][v] += bottleneck
            flow[v][u] -= bottleneck
            v = u

        max_flow += bottleneck

    reachable = set([source])
    q = deque([source])

    while q:
        u = q.popleft()
        for v in adj[u]:
            residual = capacity[u][v] - flow[u][v]
            if residual > 0 and v not in reachable:
                reachable.add(v)
                q.append(v)

    S = reachable
    all_vertices = set(adj.keys())
    for u in adj.keys():
        all_vertices.update(adj[u])
    T = all_vertices - S

    return max_flow, S, T, flow


In [ ]:
vertices = ["s", "v1", "v2", "v3", "v4", "t"]

capacity = defaultdict(lambda: defaultdict(int))
adj = defaultdict(list)

def add_edge(u, v, c):
    capacity[u][v] += c
    if v not in adj[u]:
        adj[u].append(v)
    if u not in adj[v]:
        adj[v].append(u)

add_edge("s",  "v1", 16)
add_edge("s",  "v2", 13)
add_edge("v2", "v1", 4)
add_edge("v1", "v3", 12)
add_edge("v2", "v4", 14)
add_edge("v3", "v2", 9)
add_edge("v4", "v3", 7)
add_edge("v3", "t",  20)
add_edge("v4", "t",  4)

max_flow, S, T, flow = edmonds_karp(capacity, adj, "s", "t")

print("Maximum flow :", max_flow)
print("Minimum cut :")
print("  S =", S)
print("  T =", T)
